In [ ]:
seed_value= 100
# 1. Set `PYTHONHASHSEED` environment variable at a fixed value
import os
os.environ['PYTHONHASHSEED']=str(seed_value)
# 2. Set `python` built-in pseudo-random generator at a fixed value
import random
random.seed(seed_value)
# 3. Set `numpy` pseudo-random generator at a fixed value
import numpy as np
np.random.seed(seed_value)
# 4. Set the `tensorflow` pseudo-random generator at a fixed value
import tensorflow as tf
tf.random.set_seed(seed_value)
# for later versions:
# tf.compat.v1.set_random_seed(seed_value)
# 5. Configure a new global `tensorflow` session
from keras import backend as K
# session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
# sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
# K.set_session(sess)
# for later versions:
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
tf.compat.v1.keras.backend.set_session(sess)
import keras

In [ ]:
#import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold #--needed for cross validation
from sklearn.preprocessing import StandardScaler
import pandas as pd
#import keras
import scipy as sc
from scipy import signal
import autokeras as ak
#import tensorflow as tf
from tensorflow.keras import layers
from autokeras import StructuredDataRegressor

In [ ]:
class GeoData:
    def __init__(self,file):
        self.Scaler = StandardScaler()
        self.df = pd.read_hdf(file)
        #self.df = pd.read_pickle("./SuperStack.pkl")
        print(self.df.columns) 
        X_df= self.df[["SRC_Lat","SRC_Lon","REC_Lat","REC_Lon"]].copy()
        y_df= self.df[["Greens"]].copy()
       
        
        self.X=  X_df.to_numpy()
        self.y= y_df.to_numpy()
        self.y=  self.y[:,0]
        self.y = [ x.tolist() for x in self.y]
        self.y= np.array(self.y)
        self.y= self.y/ np.max(self.y,axis=1).reshape(-1,1)
                         
        
        #self.y.shape = (380)
        print(self.y.shape)
        #print(self.y)
        print(self.X.shape)
        print(self.X[0])
        print(self.y[0])
testFile_h="./Data/Supersets/hdf/SuperStack.hdf"  
#testFile_s="SuperStack.hdf"
print("Reading in HourStacked Data...")
Geo = GeoData(testFile_h)

In [ ]:

print('-Sameness Check-')
print(Geo.y[0,:] - Geo.y[100,:])
print(Geo.X[0,:] - Geo.X[100,:])


In [ ]:
def kFolds(Geo):
    kf = KFold(n_splits=2)
    X= Geo.X.copy()
    y= Geo.y.copy()
    KFold(n_splits=2, random_state=42, shuffle=True)
    count=0
    print("Will use for Cross Validation for training")
    for train_index, test_index in kf.split(X):
        if(count==0):
            #print("Train:", train_index, "Test:", test_index)
            train_x_crossV1, test_x_crossV1 = X[train_index], X[test_index]
            train_y_crossV1, test_y_crossV1 = y[train_index], y[test_index]
            #print(train_x_crossV1)
            count+=1
        else:
            #print("Train:", train_index, "Test:", test_index)
            train_x_crossV2, test_x_crossV2 = X[train_index], X[test_index]
            train_y_crossV2, test_y_crossV2 = y[train_index], y[test_index]
    return test_x_crossV1,test_y_crossV1,test_x_crossV2,test_y_crossV2


X_train_h,y_train_h,X_test_h,y_test_h= kFolds(Geo) #for hoursStack

In [ ]:
# define the search
search = StructuredDataRegressor(max_trials=15, loss='mean_squared_error')

# perform the search/fitting to data
search.fit(x=X_train_h, y=y_train_h, verbose=0,validation_split=0.2)
search.fit(x=X_test_h, y=y_test_h, verbose=0,validation_split=0.2)

# evaluate the model
mae_h, _ = search.evaluate(X_test_h, y_test_h, verbose=0)
print('Mean Square Error Average: %.3f' % mae_h)

# get the best performing model
model_h = search.export_model()
# model_h.save('HourStackModel.h5',save_format='tf')
# summarize the loaded model
model_h.summary()
# save the best performing model to file

# model_h.compile()

model.save('model_insurance.h5')

In [ ]:
# get the best performing model
model = search.export_model()
# model_h.save('HourStackModel.h5',save_format='tf')
# summarize the loaded model
model.summary()
# save the best performing model to file

# model_h.compile()

# model.save('model_insurance.h5')

In [ ]:
model_h.predict(Geo.X[50:50+1])

model_h.predict(Geo.X[10:10+1])

In [ ]:

model_h.predict(Geo.X[10:10+1]).flatten() - model_h.predict(Geo.X[50:50+1]).flatten()
